In [3]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

from xgboost import XGBRFRegressor
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

from sklearn.utils.class_weight import compute_sample_weight

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline as imb_make_pipeline

/Users/clarissarjtai/miniforge3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [259]:
restaurant_csv_path = "data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv"
df = pd.read_csv(restaurant_csv_path)
# Convert A, B, C to 3, 2, 1 for encodings, drop all other ratings
grades = ["A", "B", "C"]
df = df[df['GRADE'].isin(grades)]
df['SCORE'] = df.loc[:, 'GRADE']
df.loc[df["SCORE"] == "A", "GRADE"] = 3
df.loc[df["SCORE"] == "B", "GRADE"] = 2
df.loc[df["SCORE"] == "C", "GRADE"] = 1

# Replace violation code, zipcode and cuisine type with its average grade:
target = df["GRADE"]
te = TargetEncoder(cols=["VIOLATION CODE", "ZIPCODE", "CUISINE DESCRIPTION"])
te.fit(df, target)
df = te.transform(df)

# Peform OHE on Boro and Critical Flag:
df = pd.get_dummies(df, columns=["BORO"])
df = pd.get_dummies(df, columns=["CRITICAL FLAG"])

print(df.columns)
df = df.drop(["DBA", 
              "BUILDING", 
              "STREET",
              "PHONE",
              "ACTION",
              "VIOLATION DESCRIPTION",
              "Latitude",
              "Longitude",
              "Community Board",
              "Council District",
              "Census Tract", 
              "RECORD DATE",
              "INSPECTION DATE",
              "INSPECTION TYPE",
              "CRITICAL FLAG_Critical",
              "BIN",
              "BBL",
              "SCORE",
              "NTA"], axis=1)
df = df.dropna()
df.head()

Index(['CAMIS', 'DBA', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'SCORE', 'GRADE', 'GRADE DATE', 'RECORD DATE',
       'INSPECTION TYPE', 'Latitude', 'Longitude', 'Community Board',
       'Council District', 'Census Tract', 'BIN', 'BBL', 'NTA', 'BORO_0',
       'BORO_Bronx', 'BORO_Brooklyn', 'BORO_Manhattan', 'BORO_Queens',
       'BORO_Staten Island', 'CRITICAL FLAG_Critical',
       'CRITICAL FLAG_Not Applicable', 'CRITICAL FLAG_Not Critical'],
      dtype='object')


,CAMIS,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,GRADE DATE,BORO_0,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Applicable,CRITICAL FLAG_Not Critical
0,41262792,2.622593,2.781846,2.910568,3,08/26/2021,0,0,0,1,0,0,0,1
2,50063071,2.793506,2.688846,2.623830,3,09/18/2018,0,0,0,1,0,0,0,0
3,40740446,2.807083,2.780662,2.488722,3,10/23/2019,0,1,0,0,0,0,0,0
4,50051826,2.586134,2.616895,2.605223,3,05/30/2017,0,0,0,0,1,0,0,0
5,50044207,2.823642,2.635659,2.910568,1,06/07/2019,0,0,1,0,0,0,0,1


In [99]:
# # Metadata for each restaurant
# meta_df = df.drop(columns=['GRADE', 'GRADE DATE', 'VIOLATION CODE','CRITICAL FLAG_Not Applicable', 'CRITICAL FLAG_Not Critical']).drop_duplicates()
# meta_df.shape

(23550, 9)

In [14]:
# check whether CAMIS in unique ID
print("Number of restaurant: ", pd.unique(df['CAMIS']).shape)
print("Dataframe shape: ", df.shape)

Number of restaurant:  (23550,)
df shape:  (159437, 14)


In [260]:
# transform to datetime object
import datetime as dt 
df['GRADE DATE'] = [dt.datetime.strptime(str(date_string), '%m/%d/%Y') for date_string in df['GRADE DATE'].values ]
# Find records after 2019
df.drop_duplicates(inplace=True)
print("Number of records after 2019: ", sum(df['GRADE DATE'] > dt.datetime.strptime("01/01/2019", '%m/%d/%Y')))
df.reset_index(drop = True, inplace= True)
df['GRADE'] = df['GRADE'].astype(float)

Number of records after 2019:  89504


In [255]:
print("Most recent grade date: ", df['GRADE DATE'].max())
print("Earliest grade date: ", df['GRADE DATE'].min())

Most recent grade date:  2022-03-15 00:00:00
Earliest grade date:  2013-06-07 00:00:00


# Average inspection by restaurant by date

In [275]:
# Average inspection by restaurant by date
df = df.groupby(['CAMIS', 'GRADE DATE']).mean().reset_index()
df.head()
print(df.shape)

(62958, 14)


# Find previous inspection

In [271]:
# Calculate recency: recent visit - previous visit
camis = df['CAMIS'].unique()
new_df = pd.DataFrame()
for idx in camis:
    # filter the records of specific restuarant using CAMIS
    subset = df[df.CAMIS == idx].sort_values(by = 'GRADE DATE')
    if subset.shape[0] == 1:
        continue
    prev_df = pd.DataFrame(index=subset.index.values)
    prev_df['GRADE DATE'] = subset['GRADE DATE']
    # prev_df['PREV_CRITICAL FLAG_Not Applicable'] = subset.shift(1)['CRITICAL FLAG_Not Applicable']
    # prev_df['PREV_CRITICAL FLAG_Not Critical'] = subset.shift(1)['CRITICAL FLAG_Not Critical']
    # find previous grading
    
    # find previous grading date and RECENCY
    df_date = subset[['GRADE', 'GRADE DATE']].drop_duplicates()
    if df_date.shape[0] != df_date['GRADE DATE'].unique().shape[0]:
        print(idx)
        break
    df_date['PREV_GRADE'] = df_date.shift(1)['GRADE']
    df_date['PREV_GRADE DATE'] = df_date.shift(1)['GRADE DATE']
    df_date['RECENCY'] = df_date['GRADE DATE'] - df_date.shift(1)['GRADE DATE']
    prev_df = prev_df.merge(df_date.drop(columns='GRADE'), how = 'left', on='GRADE DATE').set_index(subset.index.values)
    # append to new dataframe
    new_df = pd.concat([new_df, prev_df], axis=0)

# Filter rows after year 2019

In [274]:
df_concat = pd.concat([df, new_df.drop(columns=['GRADE DATE'])], axis=1)
# filter rows with date
df_concat_filtered = df_concat[df['GRADE DATE'] > dt.datetime.strptime("01/01/2019", '%m/%d/%Y')]
print(df_concat_filtered.shape)
df_concat_filtered.head(10)

(37692, 17)


,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,BORO_0,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Applicable,CRITICAL FLAG_Not Critical,PREV_GRADE,PREV_GRADE DATE,RECENCY
0,30075445,2019-06-11,2.873776,2.780662,2.848201,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.000000,NaN,NaT,NaT
1,30075445,2019-11-04,2.873776,2.780662,2.628944,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,2019-06-11,146 days
2,30075445,2021-08-10,2.873776,2.780662,2.788053,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,3.0,2019-11-04,645 days
3,30112340,2019-03-04,2.813049,2.910918,2.794176,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.666667,NaN,NaT,NaT
4,30191841,2019-06-06,2.838297,2.806115,2.860426,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.500000,NaN,NaT,NaT
5,30191841,2022-01-04,2.838297,2.806115,2.873446,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.666667,3.0,2019-06-06,943 days
6,40356018,2019-11-08,2.871212,2.838331,2.861029,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.666667,NaN,NaT,NaT
7,40356018,2022-02-01,2.871212,2.838331,2.623830,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,3.0,2019-11-08,816 days
8,40356483,2019-05-03,2.815534,2.785732,2.856327,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.666667,NaN,NaT,NaT
9,40356731,2019-05-31,2.780118,2.902107,2.852704,3.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.666667,NaN,NaT,NaT


# Null values in each column

In [273]:
df_concat_filtered.isnull().sum()

CAMIS                               0
GRADE DATE                          0
ZIPCODE                             0
CUISINE DESCRIPTION                 0
VIOLATION CODE                      0
GRADE                               0
BORO_0                              0
BORO_Bronx                          0
BORO_Brooklyn                       0
BORO_Manhattan                      0
BORO_Queens                         0
BORO_Staten Island                  0
CRITICAL FLAG_Not Applicable        0
CRITICAL FLAG_Not Critical          0
PREV_GRADE                      11169
PREV_GRADE DATE                 11169
RECENCY                         11169
dtype: int64

# Frequency of inspection for each restaurant after 2019

In [280]:
# Calculate frequency: time of grading within three year
df_freq = pd.DataFrame(df_concat_filtered.groupby(['CAMIS']).size(), columns=["frequency"])
print(df_freq.frequency.unique())
print(df_freq.head(10))

[3 1 2 4 5]
          frequency
CAMIS              
30075445          3
30112340          1
30191841          2
40356018          2
40356483          1
40356731          2
40357217          2
40359480          1
40359705          2
40360045          2


In [284]:
# Inspection is not once a year
df_concat_filtered[df_concat_filtered.CAMIS == 41319107]

,CAMIS,GRADE DATE,ZIPCODE,CUISINE DESCRIPTION,VIOLATION CODE,GRADE,BORO_0,BORO_Bronx,BORO_Brooklyn,BORO_Manhattan,BORO_Queens,BORO_Staten Island,CRITICAL FLAG_Not Applicable,CRITICAL FLAG_Not Critical,PREV_GRADE,PREV_GRADE DATE,RECENCY
11246,41319107,2019-01-28,2.682526,2.838331,2.736535,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,NaN,NaT,NaT
11247,41319107,2019-05-30,2.682526,2.838331,2.715339,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,3.0,2019-01-28,122 days
11248,41319107,2019-09-23,2.682526,2.838331,2.787974,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,3.0,2019-05-30,116 days
11249,41319107,2020-01-30,2.682526,2.838331,2.650203,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.500000,3.0,2019-09-23,129 days
11250,41319107,2022-01-27,2.682526,2.838331,2.735431,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,3.0,2020-01-30,728 days


# Output preprocessed data as csv file

In [285]:
# df_concat_filtered.to_csv("preprocessed_data/preprocessed_restaurant.csv")

In [233]:
# treat each record as a sample.
# How do we include datetime feature? Previous grade, previous grade date interval
# What if new restaurant include? Train-test split has to be in time order.

In [ ]:
# treat each restaurant as a sample.
# How do we treat ['GRADE', 'GRADE DATE', 'VIOLATION CODE','CRITICAL FLAG_Not Applicable', 'CRITICAL FLAG_Not Critical']